## Lib

In [1]:
import pandas as pd
import numpy as np

from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble

from sktime.classifiers.compose import ColumnEnsembleClassifier

from sktime.utils.load_data import from_long_to_nested
from sklearn.metrics import accuracy_score

C:\Users\delan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\delan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Data

In [2]:
full_array = np.load('data/Foch_transplant.npy', allow_pickle=True)
col = np.load('data/Foch_transplant_name_col.npy', allow_pickle=True)

In [3]:
full_array.shape

(330, 71)

In [4]:
df = pd.DataFrame(full_array, columns=col)

# Sktime

In [5]:
df.columns

Index(['Age_donor', 'Aspirations_donor', 'BMI_donor', 'Donneur_CPT',
       'Insuffisance_renale', 'LAS', 'PAPS', 'PFO', 'PF_donor', 'Poids',
       'Poids_donor', 'RX_donor', 'Sex_donor', 'Tabagisme_donor', 'Taille',
       'Taille_donor', 'age', 'body_mass_index', 'diabetes', 'id_patient',
       'other_organ_transplantation', 'oto_score', 'pathologie',
       'plasmapherese', 'preoperative_ECMO', 'preoperative_ICU',
       'preoperative_mechanical_ventilation',
       'preoperative_pulmonary_hypertension', 'preoperative_vasopressor',
       'retransplant', 'sexe', 'super_urgence', 'thoracic_surgery_history',
       'time_on_waiting_liste', 'transplanted_twice_during_study_period',
       'target', 'length_op', 'start_operation_year', 'start_operation_month',
       'start_operation_day', 'ends_operation_year', 'ends_operation_month',
       'ends_operation_day', 'B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       

In [6]:
len(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done'])

28

In [7]:
%%time
dic1 = {'case_id':[], 'dim_id':[], 'reading_id':[], 'value':[]}
nb_max = df['SpO2_by_FiO2'].apply(len).max()
for k, col in enumerate(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']):
    
    for i, arr in enumerate(df[col]):
        l = len(arr)
        for j, val in enumerate(arr):

            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [j]
            dic1['value']+= [val]
        # get ts same length
        for h in range(nb_max-l):
            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [l+h]
            dic1['value']+= [0]

Wall time: 5.98 s


In [8]:
ddff1 = pd.DataFrame(dic1)

In [ ]:
%%time
X_nested1 = from_long_to_nested(ddff1)

X_nested1.columns = ['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']

In [ ]:
X_nested1.head()

In [ ]:
X_nested1.shape

#### Sample of the base

#### 40 train 30 test

In [14]:
%%time
y_tr = df['target'][:40].ravel().astype('<U1')
y_te = df['target'][30:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:40][[col]]
    x_te = X_nested1[30:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

B.I.S : 0.46
BIS SR : 0.51
DC : 0.51
ETCO2 : 0.58
FC : 0.48
FR : 0.47
FiO2 : 0.55
PAPdia : 0.54
PAPmoy : 0.53
PAPsys : 0.5
PASd : 0.54
PASm : 0.5
PASs : 0.58
PEEPtotal : 0.46
PNId : 0.48
PNIm : 0.51
PNIs : 0.45
Pmax : 0.52
Pmean : 0.5
SpO2 : 0.65
SvO2 (m) : 0.44
Temp : 0.49
VT : 0.46
SpO2_by_FiO2 : 0.58
PASs_minus_PASd : 0.59
PAPsys_minus_PAPdia : 0.53
declampage_cote1_done : 0.58
declampage_cote2_done : 0.58
Wall time: 1h 54min 58s


#### 100 train 100 test

In [16]:
%%time
y_tr = df['target'][:100].ravel().astype('<U1')
y_te = df['target'][230:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:100][[col]]
    x_te = X_nested1[230:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

KeyboardInterrupt: 

##### Full

In [14]:
%%time
y_tr = df['target'][:230].ravel().astype('<U1')
y_te = df['target'][230:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:230][[col]]
    x_te = X_nested1[230:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

B.I.S : 0.51
BIS SR : 0.46
DC : 0.47
ETCO2 : 0.61
FC : 0.56
FR : 0.54
FiO2 : 0.52
PAPdia : 0.55
PAPmoy : 0.51
PAPsys : 0.56
PASd : 0.51
PASm : 0.52
PASs : 0.51
PEEPtotal : 0.48
PNId : 0.48
PNIm : 0.45
PNIs : 0.46
Pmax : 0.59
Pmean : 0.5
SpO2 : 0.63
SvO2 (m) : 0.47
Temp : 0.47
VT : 0.48
SpO2_by_FiO2 : 0.62
PASs_minus_PASd : 0.57
PAPsys_minus_PAPdia : 0.51
declampage_cote1_done : 0.58
declampage_cote2_done : 0.58
Wall time: 4h 17min 28s


In [ ]:
BIS SR : 0.46 * 
ETCO2 : 0.61 * 
FC : 0.56 * 
FR : 
FiO2 : 0.52 * 
PAPsys : 0.56 * 
PASm : 0.52 * 
PEEPtotal : 0.48 * 
SpO2 : 0.63 * 
SpO2_by_FiO2 : 0.62 * 